In [ ]:
import datetime
from itertools import zip_longest
import os
import re
import sys

import pandas as pd

import muscope

In [ ]:
print(sys.modules['muscope'].__file__)
muscope_loader_dp = os.path.dirname(sys.modules['muscope'].__file__)
downloads_dp = os.path.join(muscope_loader_dp, 'downloads')
dyhrman__xls_fp = os.path.join(downloads_dp, 'Dyhrman_HL2A_RNAdiel_seq_assoc_data_v3.xlsx')

print(dyhrman__xls_fp)
os.path.exists(dyhrman__xls_fp)


In [ ]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)


In [ ]:
core_attr_plus_data_df = pd.read_excel(
    dyhrman__xls_fp,
    sheet_name='core attributes + data',
    skiprows=(0,2)
)

column_headers = list(core_attr_plus_data_df.columns)
n = column_headers.index('gene_name')
column_headers[n] = 'data_type'
core_attr_plus_data_df.columns = column_headers

for (r1, row1), (r2, row2), (r3, row3), (r4, row4) in grouper(core_attr_plus_data_df.iterrows(), n=4):
    # remove .fastq.tar from sample names
    if str(row1.sample_name).endswith('.fastq.tar'):
        core_attr_plus_data_df.loc[r1, 'sample_name'] = row1.sample_name[:-10]
    else:
        pass

    if row1.data_type == 'mRNA':
        core_attr_plus_data_df.loc[r1, 'data_type'] = 'mRNA Reads'
    else:
        raise Exception()
    if row3.data_type == 'total RNA':
        core_attr_plus_data_df.loc[r3, 'data_type'] = 'Total RNA Reads'
    else:
        raise Exception()

    # append .gz to file names
    if str(row1.seq_name).endswith('.fastq'):
        core_attr_plus_data_df.loc[r1, 'seq_name'] = row1.seq_name + '.gz'
    else:
        pass
    if str(row2.seq_name).endswith('.fastq'):
        core_attr_plus_data_df.loc[r2, 'seq_name'] = row2.seq_name + '.gz'
    else:
        pass
    if str(row3.seq_name).endswith('.fastq'):
        core_attr_plus_data_df.loc[r3, 'seq_name'] = row3.seq_name + '.gz'
    else:
        pass
    if str(row4.seq_name).endswith('.fastq'):
        core_attr_plus_data_df.loc[r4, 'seq_name'] = row4.seq_name + '.gz'
    else:
        pass

    # copy attributes from previous sample ONLY IF THE ATTRIBUTE IS EMPTY
    column_names = list(core_attr_plus_data_df.columns)
    for attr_name in column_names:
        #print('row {} attr "{}" is "{}"'.format(r3, attr_name, core_attr_plus_data_df.loc[r3, attr_name]))
        if str(core_attr_plus_data_df.loc[r3, attr_name]) in ('nan', 'NaT'):
            #print('  copy "{}" from previous sample'.format(core_attr_plus_data_df.loc[r1, attr_name]))
            core_attr_plus_data_df.loc[r3, attr_name] = core_attr_plus_data_df.loc[r1, attr_name]
        else:
            pass

core_attr_plus_data_df